# BB84_Demo

In [ ]:
import numpy as np
import random

from qiskit_network.components import Network
from qiskit_network.components.storage import QuantumStorage, ClassicalStorage
from qiskit_network.channels import Channels, QuantumChannels, ClassicalChannels
from qiskit_network import logger

from qiskit.utils import QuantumInstance
from qiskit import Aer

In [ ]:
# the backend to run the circuit
qi = QuantumInstance(Aer.get_backend('aer_simulator_statevector'), shots=1)

# pennylane-qiskit plugin may use here, but not sure if plugin will include pennylane-pulse.
#pennylane = qml.device('default.qubit', wires=1)

In [ ]:
#alice will create its own network, it also can interact with other network, we should auto determind backend.
network  = Network.Environment(name='alice_net', nodes=['Alice', 'Bob','Eve'], backend=qi)  

Alice = network.get_node('Alice')
Bob = network.get_node('Bob')
# so this will be seperate out from normal setup.
# todo: set eve
network.get_node('eve').eve_interception()

# set all network with idendical setup
# this will use for storing packet message
Cstorage = ClassicalStorage()
# Not sure how quantum storage will works out with real device yet, but we want to simulate the photon components in pulse, 
# there should be some research paper for references in #14 or pennylane photon.
# todo: need more inventigation to how to tranform it into pulse level
Qstorage = QuantumStorage(name="", timeline=None, num_memories=10,fidelity=0.85, frequency=80e6, efficiency=1, coherence_time=-1, wavelength=500)

# if any settings require qiskit pulse, will automatically turn into qiskit-pulse scheduler, but it only work for clean simulator.
QKD_QC = QuantumChannels(name="", timeline=None,distance = 1000,attenuation=0, polarization_fidelity = 1.0, light_speed=2e-4, frequency=8e7) 
ethernet_CC = ClassicalChannels(name="",timeline=None, distance = 1000,delay=1e9)
#other type of channels, like service channels for other protocols
# todo: how to setup custom encryption, will need to look into https://github.com/OpenKMIP/PyKMIP/tree/master/kmip/demos
encrypted_CC = ClassicalChannels(name="encrypted data over fiber",timeline= None, distance = 1000,delay=1e9, ) 

# todo: what else need to set here, need to learn more about channels from different quantum internet Standardization and latest research
channels = Channels(Classical=[ethernet_CC,encrypted_CC], Quantum=QKD_QC, ) 
# connection can be any known type of ethernet connection or custom coupling map
# (which can be useful for vqe optimize quantum network connection, link: https://pennylane.ai/blog/2022/10/the-quantum-internet-and-variational-quantum-optimization/)
network.setup_all(storage=[Qstorage,Cstorage], channels=channels, connection='mesh') 

In [ ]:
#cascade, reference: https://github.com/upsideon/qkd-qchack-2022/blob/main/qkd/src/cascade.py


In [ ]:
# this will be action of the node whether its a receiver or a sender
class all_func:
    def __init__(self, sender, receiver, n=16):
        self.sender = sender
        self.receiver = receiver
        self.n = n

        self.bit_flips = [None for _ in range(n)]
        self.basis_flips = [random.randint(0, 1) for _ in range(n)]
        self.num_test_bits = max(n // 4, 1)
    
    def distribute_bb84_states(self, conn, epr_socket, receiver=False):
        for i in range(self.n):
            # Note that we will need to inlcude other things like bsm node, so this is where everything is differnet
            if receiver == False:
                qc = epr_socket.create_epr(num_qubit=1, post_routine= None, sequential=False, time_unit=time.SECONDS, Max_time = 0, min_fidelity = None, max_tries = None)
            else:
                qc = epr_socket.receive_epr(num_qubit=1, post_routine= None, sequential=False, min_fidelity = None, max_tries = None)
            
            if self.basis_flips[i]:
                qc.h(0)
            # todo: maybe include different measurement or settings
            m = qc.run_measure(0,0,)

            conn.flush()
            self.bit_flips[i] = int(m)
        return self.bit_flips, self.basis_flips
    
    def estimate_error_rate(self,socket,key,start = None, end = None, receiver = False):
        if receiver:
            test_indices = socket.recv_structured().payload
            start,end = test_indices
            test_outcomes = key[start:end]

            #logger.info(f"bob test indices: {test_indices}")
            #logger.info(f"bob test outcomes: {test_outcomes}")

            socket.send_structured(StructuredMessage("Test outcomes", test_outcomes))
            target_test_outcomes = socket.recv_structured().payload
        else:
            test_outcomes = key[start:end]
            test_indices = start,end

            socket.send_structured(StructuredMessage("Test indices", test_indices))
            target_test_outcomes = socket.recv_structured().payload
            socket.send_structured(StructuredMessage("Test outcomes", test_outcomes))

        num_error = 0
        for (i1, i2) in zip(test_outcomes, target_test_outcomes):
            #assert i1 == i2
            if i1 != i2:
                num_error += 1

        return (num_error / (end - start))*100
    
    def start_sender(self,start, end):
        self.start, self.end = start, end
        bit_flips, basis_flips = self.distribute_bb84_states(
            self.sender, channels.quantum(receiver = self.receiver)
        )
        #logger.info(f"sender outcomes: {bit_flips}")
        #logger.info(f"sender theta: {basis_flips}")
        socket = channels.classical(sender = self.sender, receiver = self.receiver)
        error_rate = self.estimate_error_rate(socket,bit_flips, 0, 6)
        socket.send('1' if error_rate<=0.0 else '0')
        return {
            "error_rate" : error_rate,
            "secret_key" : self.basis_flips,
        }

    def start_receiver(self):
        bit_flips, basis_flips = self.distribute_bb84_states(
            self.receiver, channels.quantum(sender = self.receiver), receriver = True
        )
        #logger.info(f"receiver outcomes: {bit_flips}")
        #logger.info(f"receiver theta: {basis_flips}")
        socket = channels.classical(sender = self.sender, receiver = self.receiver)
        error_rate = self.estimate_error_rate(socket,bit_flips, receiver = True)
        accept_string = socket.recv()
        accept_key = True if accept_string == '1' else False
        return {
            "error_rate" : error_rate,
            "secret_key" : self.basis_flips,
            "accept" :  accept_key
        }

        

In [ ]:
# this will insert in the middle of circuit
def eve(qc):
    key_string = random.randint(0, 1)
    if key_string == 0:
        qc.x(0)
    elif key_string == 1:
        qc.h(0)
    # qiskit dynamic circuit here
    qc.run_measure()

In [ ]:
# which may include more function settings, all_func will always include sender and receiver variable to run analysis or any thing.
network.set_function(all_func, interception=eve)

In [ ]:
# how the overall circuit look like
display(network.construct_circuit().draw())

In [ ]:
result = network.run()
# sample output:
"""
{"Alice": {"error_rate": 6.2, "secret_key": xxxxxx}, "Bob": {"error_rate": 6.2, "secret_key": xxxxxx, "accept" :  0}  }
"""

In [ ]:
# visualization or maybe 
result

In [ ]:
# or setup for individually
#alice = network.get_node('Alice')


# seperate for the node
#def alice_func(Alice):
#        msg_buff = []
#        distribute_bb84_states(alice, msg_buff, secret_key, network.get_node('eve'))
#        estimate_error_rate(alice,key, )
#
#def bob_func(bob):
#        msg_buff = []
#        distribute_bb84_states(bob, msg_buff, secret_key, network.get_node('eve'))
#        estimate_error_rate(bob,key, )